In [1]:
import inspect
import random
from weakref import WeakKeyDictionary

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

In [3]:
PRIMITIVE_TYPES = [str, int]       

In [4]:
class Symbolic(object):
    def __init__(self):
        self.symbols = WeakKeyDictionary()
 
    def __get__(self, instance_obj, objtype):
        return self.symbols[instance_obj]
 
    def __set__(self, instance, values):
        self.symbols[instance] = Symbol(values)
 
    def __delete__(self, instance):
        del self.symbols[instance]

In [34]:
class Symbol(object):
    def __init__(self, values):
        self.parent = None
        self.InitVals(values)
        
    def InitVals(self, values):
        if inspect.isclass(values):
            self.values = []
            for subclass in values.__subclasses__():
                self.values.append(subclass())
        else:
            self.values = values
        self._value = None
        self.CalcAttributes()
        
    def __repr__(self):
        return str(self.val)
    
    def __getattr__(self, attr):
        # allow self.val to inspect without collapsing my value
        if attr == 'val':
            if self._value:
                return self._value
            else:
                if len(self.values) == 1:
                    return self.values[0]
                else:
                    return self.values
        raise AttributeError("{} object has no attribute {}".format(self.__class__, attr))
    
    def CalcAttributes(self):
        attribute_values = {}
        for obj in self.values:
            if type(obj) in PRIMITIVE_TYPES:
                continue
            attributes = dir(obj)
            filters = [lambda x: x[0] != '_', lambda x: not callable(getattr(obj, x)), lambda x: x not in PRIMITIVE_TYPES]
            for fn in filters:
                attributes = list(filter(fn, attributes))
            for attribute in attributes:
                if attribute not in attribute_values:
                    attribute_values[attribute] = []
                val = obj.__getattribute__(attribute)
                if isinstance(val, Symbol):
                    if type(val.values) is list:
                        attribute_values[attribute].extend(val.values)
                    else:
                        attribute_values[attribute].append(val.values)
                else:
                    attribute_values[attribute].append(val)
        for key, vals in attribute_values.items():
                symbol = Symbol(vals)
                symbol.parent = self
                setattr(self, key, symbol)
    
    def Observe(self):
        if self._value:
            return self._value
        else:
            return self.Collapse(None)
    
    def Collapse(self, symbol, upwards=True):
        if symbol is None:
            self._value = random.choice(self.values)
            self.values = [self._value]
            if upwards:
                if self.parent is not None:
                    self.parent.Collapse(self.parent, upwards=True)
            return self._value
        
        new_values = subset(self, symbol)
        
        if type(new_values) is list: 
            if len(new_values) == 0:
                raise Exception('empty symbol')
            self.values = new_values
        else:
            self.values = [new_values]
        
        # recursively apply to all shared symbolic attributes
        new_values = []
        filters = [
            lambda key, value: key[0] != '_',
            lambda key, value: not callable(value),
            lambda key, value: key != 'parent' and key != 'values'
        ]
        symbol_attributes = {i: symbol.__getattribute__(i) for i in dir(symbol)}
        for fn in filters:
            symbol_attributes = {k: v for k, v in symbol_attributes.items() if fn(k, v)}
        print(symbol_attributes)
        print()
        for value in self.values:
            attribues_match = True
            
            value_attributes = {i: value.__getattribute__(i) for i in dir(value)}
            for fn in filters:
                value_attributes = {k: v for k, v in value_attributes.items() if fn(k, v)}
                
            print(value_attributes)
            print()
            for attribute in value_attributes.keys():
                if not attribues_match:
                    continue
                if attribute in symbol_attributes:
                    symbol_attribute = symbol_attributes[attribute]
                    value_attribute = value_attributes[attribute]
                    value_attribute = subset(value_attribute, symbol_attribute)
                    if value_attribute is None:
                        attribues_match = False
                    else:
                        setattr(value, attribute, value_attribute)
                else:
                    pass
                    # adopt missing target values to ensure consistency
            if attribues_match:
                new_values.append(value)
        self.values = new_values

        self.CalcAttributes()
        if upwards:
            if self.parent is not None:
                self.parent.Collapse(self.parent, upwards=True)
        return self
                 

def subset(value, target, depth=0):
    '''Return the subset of value that spans target. If no
    such subset exists, return None
    '''
#     print('\t'*depth, 'subset: {}, {}'.format(value, target))
    is_subset = False
    if type(value) is Symbol:
        return subset(value.values, target, depth=depth+1)
    if type(value) is list:
        results = []
        for element in value:
            results.append(subset(element, target, depth=depth+1))
        results = [result for result in results if result is not None]
        if len(results) == 0:
            return None
        if len(results) == 1:
            return results[0]
        return results
        
    if type(target) is Symbol:
        return subset(value, target.values, depth=depth+1)
    if type(target) is list:
        for element in target:
            result = subset(value, element, depth=depth+1)
            if result is not None:
                return result
        return None
        
    if value == target:
        is_subset = True
    if type(value) not in PRIMITIVE_TYPES:
        if type(value) == type(target):
            is_subset = True
        if type(target) in type(value).__subclasses__():
            # TODO: (subclass)Value promotion
            # just take all attributes and methods
            is_subset = True
#     print('\t'*depth, is_subset)
    if not is_subset:
        return None
    
    return value

# Basic Functionality

In [35]:
class Fruit(object):
    color = Symbolic()
    def __init__(self, name):
        self.name = name
    def method(self):
        print('method man')

class Apple(Fruit):
    def __init__(self):
        self.name = 'Apple'
        self.color = ['red', 'green']

class Pear(Fruit):
    def __init__(self):
        self.name = 'Pear'
        self.color = ['red', 'yellow']

        
class State(object):
    fruit = Symbolic()
    def __init__(self):
        self.fruit = Fruit

In [36]:
state = State()

print(state.fruit)

[<__main__.Apple object at 0x1061b2dd8>, <__main__.Pear object at 0x1061b2518>]


In [37]:
print(state.fruit.color)

['red', 'green', 'red', 'yellow']


In [38]:
print(state.fruit.color.Observe())

{'color': yellow, 'name': ['Apple', 'Pear']}

{'color': ['red', 'green'], 'name': 'Apple'}

{'color': ['red', 'yellow'], 'name': 'Pear'}

yellow


In [39]:
print(state.fruit.name)

Pear


In [40]:
print(state.fruit.name.Observe())

{'color': yellow, 'name': Pear}

{'color': yellow, 'name': 'Pear'}

Pear


In [41]:
print(state.fruit)

In [42]:
print(state.fruit.name)
print(state.fruit.color)

Pear
yellow


# First attempt: Simple Collapse

Object is considered a subset of another if it is the same type or a subclass of the other's type. 

### Collapse
    1. restrict my values to the minimal subset of target values
        if I am None, return None
    2. for each of my values, for each of shared attributes 
        restrict(value.attr, target.attr)
        if they return None, I return None
    2.5. recalculate my attributes
    3. adopt all other attributes of the target values
    4. if i am the first, collapse upward
    
### Collapse upward
    2. for each of my values, if they have attribute
        restrict value attributes to the target attributes.
        if they return None, drop this value
    if i have lost all of my values, throw exception
    if i have a parent, collapse upwards
        
### Restriction
    for every element in me, keep if i am a superset of every element in target.
        this ensures that we can take any possible instance of the target and find its 
        realization in original values
        
### Subset
    I am a subset of the target value if I am the same class or a subclass.
    


### Consistency condition
    new value must always be a strict subset of the existing values:
    1. New value implements every attribute of the parent object, at least potentially
    2. New value implements every method of the parent object
    3. 

make symbols subscriptable

In [6]:
set([1]) | set([2])

{1, 2}